In [3]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.linalg import eigvals
import random

In [110]:
def create_weighted_cycle_graph_adj_matrix(n, weight_range=(-10, 10)):
    """
    Create a weighted directed cyclic graph with n nodes.
    
    Parameters:
    - n: The number of nodes in the graph.
    - weight_range: A tuple (min_weight, max_weight) specifying the range of weights.
    
    Returns:
    - A numpy array representing the weighted adjacency matrix of the graph.
    """
    # Initialize an n x n adjacency matrix with zeros
    adjacency_matrix = np.zeros((n, n), dtype=float)
    
    # Random weight for each edge
    for i in range(n):
        weight = np.random.randint(weight_range[0], weight_range[1] + 1)
        adjacency_matrix[i][(i + 1) % n] = weight
    
    return adjacency_matrix/10

def matrix_power(matrix, power):
    """
    Calculates the power of a matrix.

    Parameters:
    matrix (numpy.ndarray): The matrix to calculate the power of.
    power (int): The power to raise the matrix to.

    Returns:
    numpy.ndarray: The resulting matrix.
    """
    return np.linalg.matrix_power(matrix, power)

def matrix_vector_multiplication(matrix, vector):
    """
    Calculates the multiplication of a matrix and a vector.

    Parameters:
    matrix (numpy.ndarray): The matrix to multiply.
    vector (numpy.ndarray): The vector to multiply.

    Returns:
    numpy.ndarray: The resulting vector.
    """
    return np.dot(matrix, vector)


def print_multiplication_until_n(G, q, n):
    """
    Prints the multiplication of matrix and vector for k = 1 to n.

    Parameters:
    G (numpy.ndarray): The matrix to multiply.
    q (numpy.ndarray): The vector to multiply.
    n (int): The number of times to multiply the matrix and vector.

    Returns:
    None
    """
    result = np.eye(len(q))
    for i in range(1, n+1):
        result += (-1)**i * matrix_power(G, i)
        output_k = matrix_vector_multiplication(np.linalg.inv(result), q)
        output_k = np.maximum(output_k, 0)
        print("k = {}: {}".format(i, output_k))

In [88]:
# print("Infinity norm of G: " + str(np.linalg.norm(G, ord=np.inf)))
# print("Norm-2 of G: " + str(np.linalg.norm(G)))
# print("Norm-1 of G: " + str(np.linalg.norm(G, ord=1)))
# # print sorted eigenvalues of G
# print("Eigenvalues of G: " + str(sorted(eigvals(G))))

In [100]:
n = 6
G = create_weighted_cycle_graph_adj_matrix(n)
result = nx.adjacency_matrix(G, weight='weight')
result = result.todense()
q = np.ones(n)

/var/folders/hs/y116vr4j4yg5fcthw_l99c5r0000gn/T/ipykernel_3356/432110732.py:3: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  result = nx.adjacency_matrix(G, weight='weight')


In [101]:
print_multiplication_until_n(result, q, 40)

k = 1: [ 3.02001949  4.61925291 11.8852366  26.03341739 24.49169583  7.03084462]
k = 2: [1.19260039 1.13813644 1.1599389  1.02590638 0.96250365 1.20600122]
k = 3: [ 2.07975732  2.85645434  6.50535844 13.52574449 12.71242903  4.09829733]
k = 4: [1.21120492 1.15521515 1.19376434 1.08979951 1.01397855 1.23661653]
k = 5: [1.78638981 2.28563669 4.72596722 9.36536152 8.79456265 3.13914623]
k = 6: [1.21638263 1.16435064 1.22039839 1.15463364 1.07155735 1.25266705]
k = 7: [1.64231736 2.00285465 3.84123032 7.29538284 6.84538177 2.66341454]
k = 8: [1.22068577 1.17297098 1.24652708 1.2177993  1.12908015 1.267268  ]
k = 9: [1.55662616 1.83441365 3.31400872 6.06185229 5.68386092 2.38003658]
k = 10: [1.22486169 1.18135414 1.27220149 1.27921075 1.18564231 1.28138298]
k = 11: [1.49999983 1.72307992 2.96551948 5.24650722 4.91611654 2.19273719]
k = 12: [1.22893259 1.1894861  1.29725349 1.33876734 1.2408335  1.29506482]
k = 13: [1.45996643 1.64436723 2.71913756 4.67005978 4.37332309 2.06031762]
k = 14: [

In [134]:
def compute_series_matrix(G, max_degree=3):
    """Computes the series G + G^2 + G^3 + ... up to G^max_degree."""
    N = G.shape[0]
    series_matrix = np.eye(N)  # Start with I
    G_power = np.eye(N)  # Initialize with I for the first term G^0
    for n in range(1, max_degree + 1):
        G_power = np.dot(G_power, G)  # Compute G^n
        series_matrix += G_power  # Add or subtract the term based on n
    return series_matrix

def find_nash_equilibrium(G, tolerance=0.01, max_iterations=1000):
    N = G.shape[0]  # Number of agents
    x = np.ones(N)  # Initialize efforts with random values
    
    for iteration in range(max_iterations):
        x_new = np.ones(N) - np.dot(G, x)  # Compute new x 
        x_new = np.maximum(0, x_new)  # Apply the max{0, x} condition

        # print x and x_new for the last three iterations
        if iteration >= max_iterations - 3:
            print(f"Iteration {iteration + 1}: x = {x}, x_new = {x_new}")
                    
        # Check for convergence
        if np.linalg.norm(x - x_new) < tolerance:
            print(f"Equilibrium found after {iteration + 1} iterations.")
            return x_new
        
        x = x_new  # Update x for the next iteration
    
    print("Max iterations reached without convergence.")
    return x
# Example usage
N = 3  # Adjust based on your specific game model
adj_matrix = np.array([[0, -0.4, 0], [0, 0, 0.2], [0.6, 0, 0]])  # Example G matrix

# Try different degrees of the series
for max_degree in range(1, 2):  # Adjust the range as needed
    result = compute_series_matrix(adj_matrix, max_degree)
    x_star = find_nash_equilibrium(result)
    print(f"Nash Equilibrium for degree {max_degree}:", x_star)

Iteration 998: x = [1.36571429 0.08571429 1.        ], x_new = [0.         0.71428571 0.        ]
Iteration 999: x = [0.         0.71428571 0.        ], x_new = [1.28571429 0.28571429 1.        ]
Iteration 1000: x = [1.28571429 0.28571429 1.        ], x_new = [0.         0.51428571 0.        ]
Max iterations reached without convergence.
Nash Equilibrium for degree 1: [0.         0.51428571 0.        ]


In [ ]:
# Nash Equilibrium for degree 1: [0.         0.51428571 0.        ]